In [1]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import re
import string
import json
import pickle

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [46]:
from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests
from urllib2 import urlopen

In [82]:
# import pip

# def install(package):
#    pip.main(['install', package])

# install('python-mal')

# import myanimelist.session
# #username="teamvulpix", password="2015harvardcs109"
# session = myanimelist.session.Session()
# session.login()
# bebop = session.anime(20)



#### Scrapping the search bar of www.anime-planet.com

In [113]:
#48 animes per page with 152 pages = ~7296
anime = []
for i in xrange(1,153):
    anime.append(requests.get('http://www.anime-planet.com/anime/all?sort=title&order=asc&page='+str(i)))
    time.sleep(1)

#### Saving our anime pages into a pickle file

In [151]:
# Save anime into a pickle file.

# pickle.dump(anime, open("anime_pages.p", "wb"))

test1 = open("anime_pages.p",'rb') 
anime = pickle.load(test1)  
test1.close()

#### Finding all anime links

In [156]:
root = 'http://www.anime-planet.com'

all_anime = []
for page in anime:
    soup = BeautifulSoup(page.text, "html.parser")
    animelist = soup.find('ul',attrs={"class": "cardDeck pure-g cd-narrow"}).find_all("li")
    for ani in animelist:
        all_anime.append(root + ani.find('a').get('href'))

In [147]:
len(all_anime)

7281

#### Requesting each anime page information

In [227]:
each_anime = []

for i in xrange(len(all_anime)):
    each_anime.append(requests.get(all_anime[i]))
    time.sleep(1)

#### Saving each anime page

In [236]:
# Save anime into a pickle file.
# import pickle

#pickle.dump(each_anime, open("each_anime.p", "wb"))

test2 = open("each_anime.p",'rb') 
each_anime = pickle.load(test2)  
test2.close()

#### Getting each anime info

In [275]:
each_anime_info = {}

for anim in each_anime:
    anime_info = {}
    soup1 = BeautifulSoup(anim.text, "html.parser")
    name = soup1.find('h1',attrs={"itemprop": "name"}).contents[0]
    altname = soup1.find('h2',attrs={"class": "aka"})

    infos = soup1.find_all('div',attrs={"class": "pure-u-1 pure-u-md-1-5"})
    atype = infos[0]
    studio = infos[1].a
    date = infos[2].li
    date_pub = infos[2].find('span',attrs={'itemprop':'datePublished'})
    rating = infos[3].div.span
    rank = infos[4]
    description = soup1.find('div',attrs={"itemprop": "description"})
    
    #get categories
    categories_table = soup1.find('div',attrs={"class": "categories"})
    if categories_table is not None:
        anime_info['categories'] = [i.a.contents[0] for i in categories_table.ul]
    
    #get related media
    related_table = soup1.find('table',attrs={"class": "pure-table pure-table-striped noHeader"})
    if related_table is not None:
        anime_info['related'] = [row.td.a.contents[0] for row in related_table.find_all('tr')]
    
    anime_info['name'] = name
    if altname is not None:
        anime_info['altname'] = altname.contents[0]
    else:
        anime_info['altname'] = ''
    if atype is not None:
        anime_info['atype'] = atype.contents[0]
    else:
        anime_info['atype'] = ''
    if studio is not None:
        anime_info['studio'] = studio.contents[0]
    else:
        anime_info['studio'] = ''
    if date is not None:
        anime_info['date'] = date.a.contents[0]
    else:
        anime_info['date'] = ''
    if date_pub is not None:
        anime_info['date_pub'] = date_pub.a.contents[0]
    else:
        anime_info['date'] = ''
    if rating is not None:
        anime_info['rating'] = rating.contents[0]
    else:
        anime_info['rating'] = ''
    if rank is not None:
        anime_info['rank'] = rank.contents[0]
    else:
        anime_info['rank'] = ''
    if description is not None :
        if description.p is not None:
            anime_info['description'] = description.p.contents[0].string
        else:
                anime_info['description'] = ''
    else:
        anime_info['description'] = ''
    
    if name in each_anime_info.keys():
        name+='_2'
        
    each_anime_info[name] = anime_info

In [277]:
#  with open('each_anime_info.json', 'w') as fp:
#      json.dump(each_anime_info, fp)

with open('each_anime_info.json','r') as data_file:    
    each_anime_info = json.load(data_file)

In [278]:
len(each_anime_info)

7281